# Primer notebook -
En este primer notebook se definiran conceptos claves para el resto del proyecto, tales como el dataset seleccionado, rutas de las principales carpetas para la comunicacion entre notebooks, tambien el datset que se utilizara necesita generar archivos .json para el tratamiento reducido del dataset, se utilizo de base el dataset coco2017 que puede ser decargo de manera publica mediante el sigueinte link: https://www.kaggle.com/datasets/awsaf49/coco-2017-dataset
Para este proyecto se usara ya el dataste descrgado de manera local por ende no se descargara en este proyecto, para probarlo deberas descargar el dataset del link anterior y agregarlo en la carptea data de este proyecto IA-final/data/archive/coco2017
## Objetivos
1. Verificar la estructura del dataset COCO2017 en: `data\archive\coco2017`
2. Confirmar que los archivos y carpetas necesarias existen.
3. Cargar el archivo de anotaciones (instances_train2017.json / instances_val2017.json).
4. Definir 3 clases objetivo (para entrenamiento eficiente en CPU).
5. Guardar un `labelmap.json` con las clases seleccionadas.


In [1]:
"""
- Importa librerías estándar usadas en todo el notebook.
"""

import json
import os
from pathlib import Path
from collections import Counter


1. Creacion y validacion de rutas y carpetas escenciales para el proyecto

In [2]:
"""
En esta primera seecion:
- Define las rutas del dataset COCO2017 de forma única y centralizada.
- Define clases objetivo fijas: person, car, airplane. Se elegirieron estas clases dado que presentan un gran numero de referentes en el datset
- Crea la carpeta data/processed si no existe.
- Guarda el path absoluto (resolve) para evitar ambigüedades en notebooks.
"""
PROJECT_ROOT = Path("..")     

COCO_ROOT = PROJECT_ROOT / "data" / "archive" / "coco2017"

TRAIN_DIR = (COCO_ROOT / "train2017").resolve()
VAL_DIR = (COCO_ROOT / "val2017").resolve()
ANN_DIR = (COCO_ROOT / "annotations").resolve()

TRAIN_ANN = (ANN_DIR / "instances_train2017.json").resolve()
VAL_ANN = (ANN_DIR / "instances_val2017.json").resolve()

PROCESSED_DIR = (PROJECT_ROOT / "data" / "processed").resolve()
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

PROJECT_CONFIG_PATH = (PROCESSED_DIR / "project_config.json").resolve()
LABELMAP_PATH = (PROCESSED_DIR / "labelmap.json").resolve()

TARGET_CLASSES = ["person", "car", "airplane"]

print("Para que confirmes que los datos mas improtantes se encuentren tras tu descarga se debe confirmar que todos estos archvios existan")
print("TRAIN_DIR:", TRAIN_DIR)
print("VAL_DIR  :", VAL_DIR)
print("ANN_DIR  :", ANN_DIR)
print("TRAIN_ANN:", TRAIN_ANN)
print("VAL_ANN  :", VAL_ANN)
print("PROCESSED_DIR:", PROCESSED_DIR)
print("Clases seleccionadas:", TARGET_CLASSES)


Para que confirmes que los datos mas improtantes se encuentren tras tu descarga se debe confirmar que todos estos archvios existan
TRAIN_DIR: C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\train2017
VAL_DIR  : C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\val2017
ANN_DIR  : C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\annotations
TRAIN_ANN: C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\annotations\instances_train2017.json
VAL_ANN  : C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\annotations\instances_val2017.json
PROCESSED_DIR: C:\Users\Johnny\Desktop\IA-final\data\processed
Clases seleccionadas: ['person', 'car', 'airplane']


In [3]:
"""
Esta celda:
Una simple validacion de las carpetas anteriores para evitar 
conflictos de rutas en un futuro, para evitar problemas las declaramos desde el primer notebok
"""

def assert_exists(path: Path, label: str) -> None:
    if not path.exists():
        raise FileNotFoundError(f"Recurso requerido no encontrado: {label}: {path}")
    print(f"OK: {label}: {path}")

assert_exists(COCO_ROOT, "COCO_ROOT")
assert_exists(TRAIN_DIR, "train2017")
assert_exists(VAL_DIR, "val2017")
assert_exists(ANN_DIR, "annotations")
assert_exists(TRAIN_ANN, "instances_train2017.json")
assert_exists(VAL_ANN, "instances_val2017.json")


OK: COCO_ROOT: ..\data\archive\coco2017
OK: train2017: C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\train2017
OK: val2017: C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\val2017
OK: annotations: C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\annotations
OK: instances_train2017.json: C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\annotations\instances_train2017.json
OK: instances_val2017.json: C:\Users\Johnny\Desktop\IA-final\data\archive\coco2017\annotations\instances_val2017.json


2. Descripcion del dataset

In [4]:
"""
Descripcion del dataset y creacion de json
- Carga el JSON de anotaciones de train.El cual es necesario para la optimizacion del dataset
- Extrae categorías y crea mapeos id<->name.
- Verifica que las clases objetivo existan en COCO.
"""

with open(TRAIN_ANN, "r", encoding="utf-8") as f:
    train_data = json.load(f)

categories = train_data["categories"]
images = train_data["images"]
annotations = train_data["annotations"]

cat_id_to_name = {c["id"]: c["name"] for c in categories}
cat_name_to_id = {c["name"]: c["id"] for c in categories}

missing = [c for c in TARGET_CLASSES if c not in cat_name_to_id]
if missing:
    raise ValueError(
        "Estas clases objetivo no existen en COCO categories: " + ", ".join(missing)
    )

target_cat_ids = [cat_name_to_id[c] for c in TARGET_CLASSES]

print("Train JSON cargado")
print("Categorias:", len(categories))
print("Imagenes  :", len(images))
print("Anotaciones:", len(annotations))
print("TARGET_CLASSES:", TARGET_CLASSES)
print("target_cat_ids:", target_cat_ids)


Train JSON cargado
Categorias: 80
Imagenes  : 118287
Anotaciones: 860001
TARGET_CLASSES: ['person', 'car', 'airplane']
target_cat_ids: [1, 3, 5]


In [5]:
"""
- Cuenta cuántas anotaciones hay en train para cada clase objetivo.
- Sirve como sanity check de que hay suficientes datos, cabe aclarar que por tema de recursos solo una fraccion de este dataset fue usado para el entrenamiento
"""

counter = Counter()
for ann in annotations:
    cid = ann["category_id"]
    if cid in set(target_cat_ids):
        counter[cid] += 1

print("Anotaciones por clase objetivo (train):")
for cls_name, cid in zip(TARGET_CLASSES, target_cat_ids):
    print(f"{cls_name}: {counter[cid]}")


Anotaciones por clase objetivo (train):
person: 262465
car: 43867
airplane: 5135


In [6]:
"""
Aqui empezaos a crear codigo reutilizable para el resto de notebooks.
- Guarda una configuración única del proyecto con rutas absolutas.
- La idea es que todos los notebooks lean este archivo y no vuelvan a definir rutas a mano.
"""

project_config = {
    "project_root": str(PROJECT_ROOT.as_posix()),
    "coco_root": str(COCO_ROOT.as_posix()),
    "train_dir": str(TRAIN_DIR.as_posix()),
    "val_dir": str(VAL_DIR.as_posix()),
    "ann_dir": str(ANN_DIR.as_posix()),
    "train_ann": str(TRAIN_ANN.as_posix()),
    "val_ann": str(VAL_ANN.as_posix()),
    "processed_dir": str(PROCESSED_DIR.as_posix()),
    "target_classes": TARGET_CLASSES,
}

with open(PROJECT_CONFIG_PATH, "w", encoding="utf-8") as f:
    json.dump(project_config, f, indent=2, ensure_ascii=False)

print("Guardado:", PROJECT_CONFIG_PATH)


Guardado: C:\Users\Johnny\Desktop\IA-final\data\processed\project_config.json


In [7]:
"""
En esta seccion se realiza lo siguiente
- Guarda un labelmap mínimo y estable para el proyecto.
- Incluye:
  - dataset
  - target_classes
  - name_to_id e id_to_name solo para las clases objetivo
  - target_category_ids
"""

labelmap = {
    "dataset": "COCO2017",
    "target_classes": TARGET_CLASSES,
    "target_category_ids": target_cat_ids,
    "name_to_id": {name: cat_name_to_id[name] for name in TARGET_CLASSES},
    "id_to_name": {str(cat_name_to_id[name]): name for name in TARGET_CLASSES},
}

with open(LABELMAP_PATH, "w", encoding="utf-8") as f:
    json.dump(labelmap, f, indent=2, ensure_ascii=False)

print("Guardado:", LABELMAP_PATH)
print(labelmap)


Guardado: C:\Users\Johnny\Desktop\IA-final\data\processed\labelmap.json
{'dataset': 'COCO2017', 'target_classes': ['person', 'car', 'airplane'], 'target_category_ids': [1, 3, 5], 'name_to_id': {'person': 1, 'car': 3, 'airplane': 5}, 'id_to_name': {'1': 'person', '3': 'car', '5': 'airplane'}}


In [8]:
"""
Esta celda:
- Relee project_config.json y labelmap.json para asegurar que se guardaron bien.
- Esto evita que el notebook ver como si el notebook funciona, pero no guarda nada.
"""

with open(PROJECT_CONFIG_PATH, "r", encoding="utf-8") as f:
    cfg_check = json.load(f)

with open(LABELMAP_PATH, "r", encoding="utf-8") as f:
    lm_check = json.load(f)

print("project_config.json OK:", cfg_check["coco_root"])
print("labelmap.json OK:", lm_check["target_classes"])


project_config.json OK: ../data/archive/coco2017
labelmap.json OK: ['person', 'car', 'airplane']
